In [ ]:
import os
import csv
import matplotlib.pyplot as plt
from lxml import etree
from collections import defaultdict

# Function to parse TEI XML and extract relevant information
def parse_tei(xml_path):
    tree = etree.parse(xml_path)
    ns = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Extract the title of the document
    document_title = tree.xpath('//tei:title[@type="main"]/text()', namespaces=ns)[0]

    # Collect information about each speaker and their spoken words
    speaker_words = defaultdict(int)

    # Dictionary to store gender information for each person
    person_gender = {}
    for person in tree.xpath('//tei:person[@sex="FEMALE"]', namespaces=ns):
        person_id = person.xpath('@xml:id', namespaces=ns)[0]
        person_gender[person_id] = 'female'

    for sp in tree.xpath('//tei:sp', namespaces=ns):
        speaker_id = sp.xpath('.//@who', namespaces=ns)[0].lstrip('#')  # Remove initial #
        person_gender_info = person_gender.get(speaker_id)

        # Check if the speaker is female
        if person_gender_info and person_gender_info == 'female':
            words = ' '.join(sp.xpath('.//text()', namespaces=ns)).split()
            speaker_words[(speaker_id, xml_path, document_title)] += len(words)

    return speaker_words

# Main script
corpus_path = "results"

# Initialize a dictionary to store total words spoken by each female character
total_words_by_female_character = defaultdict(int)

# Process each XML document in the corpus
for xml_file in os.listdir(corpus_path):
    if xml_file.endswith(".xml"):
        xml_path = os.path.join(corpus_path, xml_file)
        speaker_words = parse_tei(xml_path)

        # Sum words spoken by each female character
        for speaker, words in speaker_words.items():
            total_words_by_female_character[speaker] += words

# Sort female characters by total words spoken in descending order
sorted_female_characters = sorted(total_words_by_female_character.items(), key=lambda x: x[1], reverse=True)

# Get the top 20 female characters
top_20_female_characters = sorted_female_characters[:20]

# Save results to a CSV file
csv_filename = "output/csv/top_20_female_characters_with_title_and_file.csv"
csv_path = os.path.join(os.getcwd(), csv_filename)

with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Character Name', 'Total Words', 'Document Title', 'File Name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for (character_id, xml_path, document_title), total_words in top_20_female_characters:
        writer.writerow({'Character Name': character_id, 'Total Words': total_words, 'Document Title': document_title, 'File Name': os.path.basename(xml_path)})

print(f"CSV file '{csv_filename}' has been created in the '{os.getcwd()}' folder.")

# Extract data for plotting
female_characters = [f"{character_id} - {document_title}" for (character_id, _, document_title), total_words in top_20_female_characters]
total_words_female = [total_words for (character_id, _, _), total_words in top_20_female_characters]

# Create a bar diagram
plt.figure(figsize=(10, 6))
plt.bar(female_characters, total_words_female, color='greenyellow')
plt.xlabel('Character Name - Title')
plt.ylabel('Total Words')
plt.title('Top 20 Female Characters with the Most Words Spoken')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save the bar diagram as a PNG file
plt.savefig('output/img/top_20_female_tragedies_characters_plot.png')

# Show the plot
plt.show()
